# Modelo preditivo

Com base nas análises realiadas nos arquivos anteriores, as variáveis que selecionamos para compor este modelo preditivo foram:

* ind_var5;
* ind_var8_0;
* ind_var30;
* num_var5;
* num_var30;
* num_var42;
* num_meses_var5_ult3;
* var36;
* var15.

In [1]:
#libs
import pandas as pd
import numpy as np

#Padronização
from sklearn.preprocessing import StandardScaler

#Automação do processo
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

#Train-test split
from sklearn.model_selection import train_test_split

#Modelos a testar
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

#Otimização de hiperparâmetros
from sklearn.model_selection import GridSearchCV

#Métricas de avaliação
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score

In [2]:
#Carregando dados
colnames = ['ind_var5','ind_var8_0','ind_var30','num_var5','num_var30','num_var42','num_meses_var5_ult3','var36','var15','TARGET']
df_train = pd.read_csv('../input/train.csv', usecols=colnames)

## Pré-processamento

Para este modelo, será necessário apenas colocar as variáveis selecionadas em escalas semelhantes

In [3]:
#Automatizamos o processo com Pipeline e ColumnTransformer
col_prep = ColumnTransformer(transformers=[('Standardize', StandardScaler(), colnames[:-1])])

def exec_pipe(model, X, y):
    model_pipe = Pipeline(steps=[('Prep', col_prep),
                                 ('Model', model)])
    return model_pipe.fit(X, y)

## Separando dados de treino e validação

Para treinar o modelo preditivo, precisamos separar um dataset balanceado entre clientes satisfeitos e insatisfeitos.

Após o treino, fazemos a validação dos dados apresentando valores não observados anteriormente.

In [4]:
#Separando o dataset balanceado para treino e validação
unsatisfied_index = df_train[df_train.TARGET > 0].index
satisfied_index = df_train[df_train.TARGET == 0].sample(len(unsatisfied_index), random_state = 5).index
train_sample = pd.concat([df_train.iloc[satisfied_index],df_train.iloc[unsatisfied_index]], axis=0)

X_train, X_valid, y_train, y_valid = train_test_split(train_sample.drop('TARGET', axis=1), train_sample.TARGET.ravel(),
                                                      test_size=0.3, train_size=0.7, random_state=7)

### Modelo 1 - Random Forest Classifier (RF)

In [5]:
rf_baseline_model = RandomForestClassifier(n_estimators=100, random_state=11)

rf_params = {'max_depth': [1,2,3,5],
             'min_samples_leaf': [1,5,10,50]}

rf_gridcv = GridSearchCV(rf_baseline_model, rf_params,
                         scoring='accuracy', cv=5, n_jobs=3)

rf_model = exec_pipe(rf_gridcv, X_train, y_train)

/home/gustavo/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/gustavo/anaconda3/lib/python3.7/site-packages/sklearn/base.py:467: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)


In [6]:
rf_preds = rf_model.predict(X_valid)
print(confusion_matrix(y_valid, rf_preds))
print(accuracy_score(y_valid, rf_preds))

[[682 235]
 [255 633]]
0.7285318559556787


/home/gustavo/anaconda3/lib/python3.7/site-packages/sklearn/pipeline.py:605: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  res = transformer.transform(X)


### Modelo 2 - Support Vector Classifier (SVC)

In [7]:
svc_baseline_model = SVC(random_state=11)

svc_params = {'C': [0.01, 0.1, 1, 10, 100]}

svc_gridcv = GridSearchCV(svc_baseline_model, svc_params,
                          scoring='accuracy', cv=5, n_jobs=3)

svc_model = exec_pipe(svc_gridcv, X_train, y_train)

/home/gustavo/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/gustavo/anaconda3/lib/python3.7/site-packages/sklearn/base.py:467: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)


In [8]:
svc_preds = svc_model.predict(X_valid)
print(confusion_matrix(y_valid, svc_preds))
print(accuracy_score(y_valid, svc_preds))

[[649 268]
 [233 655]]
0.7224376731301939


/home/gustavo/anaconda3/lib/python3.7/site-packages/sklearn/pipeline.py:605: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  res = transformer.transform(X)


### Modelo 3 - Logistic Regression (LR)

In [9]:
lr_baseline_model = LogisticRegression(multi_class='ovr', random_state=11)

lr_params = {'C': [0.01, 0.1, 1, 10, 100]}

lr_gridcv = GridSearchCV(lr_baseline_model, lr_params,
                         scoring='accuracy', cv=5, n_jobs=3)

lr_model = exec_pipe(lr_gridcv, X_train, y_train)

/home/gustavo/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/gustavo/anaconda3/lib/python3.7/site-packages/sklearn/base.py:467: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
/home/gustavo/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [10]:
lr_preds = lr_model.predict(X_valid)
print(confusion_matrix(y_valid, lr_preds))
print(accuracy_score(y_valid, lr_preds))

/home/gustavo/anaconda3/lib/python3.7/site-packages/sklearn/pipeline.py:605: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  res = transformer.transform(X)


[[580 337]
 [216 672]]
0.6936288088642659


## Separando os dados para teste

Após o treino e validação, selecionamos aleatoriamente um novo conjunto de dados para validar a performance dos modelos

In [11]:
df_test = df_train.sample(n=30000, random_state=9)

In [12]:
rf_test_preds = rf_model.predict(df_test)
print(confusion_matrix(df_test.TARGET, rf_test_preds))
print(accuracy_score(df_test.TARGET, rf_test_preds))

/home/gustavo/anaconda3/lib/python3.7/site-packages/sklearn/pipeline.py:605: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  res = transformer.transform(X)


[[22241  6535]
 [  364   860]]
0.7700333333333333


In [13]:
svc_test_preds = svc_model.predict(df_test)
print(confusion_matrix(df_test.TARGET, svc_test_preds))
print(accuracy_score(df_test.TARGET, svc_test_preds))

/home/gustavo/anaconda3/lib/python3.7/site-packages/sklearn/pipeline.py:605: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  res = transformer.transform(X)


[[21185  7591]
 [  330   894]]
0.7359666666666667


In [14]:
lr_test_preds = lr_model.predict(df_test)
print(confusion_matrix(df_test.TARGET, lr_test_preds))
print(accuracy_score(df_test.TARGET, lr_test_preds))

[[18921  9855]
 [  313   911]]
0.6610666666666667


/home/gustavo/anaconda3/lib/python3.7/site-packages/sklearn/pipeline.py:605: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  res = transformer.transform(X)


Os modelos RF e SVC apresentaram uma performance melhor quando testados com novos dados, sugerindo que não sofreram *Overfitting* durante o treinamento. Contrário a estes dois, o modelo LR apresentou uma performance pior, sugerindo que os dados selecionados não são linearmente separáveis.

Observando a matriz de confusão, percebemos que o maior problema dos três modelos está na quantidade de falsos positivos (clientes satisfeitos classificados como insatisfeitos).

Dos três modelos concebidos, RF e SVC apresentaram a melhor performance. Selecionarei estes dois para realizar previsões nos dados de testes e submeter o resultados ao Kaggle para avaliar a performance na competição.

# Submetendo modelos ao Kaggle

In [15]:
kaggle_submission_data = pd.read_csv('../input/test.csv', usecols=colnames[:-1])

In [16]:
#Realizando previsões
rf_kaggle_preds = rf_model.predict(kaggle_submission_data)
svc_kaggle_preds = svc_model.predict(kaggle_submission_data)

/home/gustavo/anaconda3/lib/python3.7/site-packages/sklearn/pipeline.py:605: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  res = transformer.transform(X)
/home/gustavo/anaconda3/lib/python3.7/site-packages/sklearn/pipeline.py:605: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  res = transformer.transform(X)


In [17]:
#Gravando resultados
sample_submission = pd.read_csv('../input/sample_submission.csv')

rf_sample_submission = sample_submission.copy()
rf_sample_submission.TARGET = rf_kaggle_preds
rf_sample_submission.to_csv('../output/rf_sample_submission.csv', index=False)

svc_sample_submission = sample_submission.copy()
svc_sample_submission.TARGET = svc_kaggle_preds
svc_sample_submission.to_csv('../output/svc_sample_submission.csv', index=False)

Scores: (Public/Private)

* RF: 0.72286 / 0.70209
* SVC: 0.72622 / 0.70815